# CAO Points Analysis

***

In [1]:
# Convenient HTTP requests.
import requests as rq

# Regular expressions.
import re

# Dates and times.
import datetime as dt

# Data frames.
import pandas as pd

# For downloading.
import urllib.request as urlrq

<br>

## 2021 Points

http://www.cao.ie/index.php?page=points&p=2021

***

In [2]:
# Fetch the CAO points URL.
resp = rq.get('http://www2.cao.ie/points/l8.php')

# Have a quick peek. 200 means OK.
resp

<Response [200]>

<br>

## Save original data set

***

In [3]:
# Get the current date and time.
now = dt.datetime.now()

# Format as a string.
nowstr = now.strftime('%Y%m%d_%H%M%S')

In [4]:
# Create a file path for the original data.
path = 'data/cao2021_' + nowstr + '.html'

<br>

## Error on server

***

Technically, the server says we should decode as per:
    
```
Content-Type: text/html; charset=iso-8859-1
```

However, one line uses \x96 which isn't defined in iso-8859-1.

Therefore we use the similar decoding standard cp1252, which is very similar but includes #x96.

In [5]:
# The server uses the wrong encoding, fix it.
original_encoding = resp.encoding

# Change to cp1252.
resp.encoding = 'cp1252'

In [6]:
# Save the original html file.
with open(path, 'w') as f:
    f.write(resp.text)

<br>

## Use regular expressions to select lines we want

***

In [7]:
# Compile the regular expression for matching lines.
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *')

<br>

#### Loop through the lines of the response

***

In [8]:
# The file path for the csv file.
path = 'data/cao2021_csv_' + nowstr + '.csv'

# Keep track of how many courses we process.
no_lines = 0

# Open the csv file for writing.
with open(path, 'w') as f:
    # Loop through lines of the response.
    for line in resp.iter_lines():
        # Decode the line, using the wrong encoding!
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_course.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            # Split the line on two or more spaces.
            linesplit = re.split('  +', dline)
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')

# Print the total number of processed lines.
print(f"Total number of lines is {no_lines}.")

Total number of lines is 922.


<br>

## 2020 Points

https://www.cao.ie/index.php?page=points&p=2020

***

<br>

#### Save Original File

***

In [9]:
# Create a file path for the original data.
path = 'data/cao2020_' + nowstr + '.xlsx'

In [10]:
urlrq.urlretrieve('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', path)

('data/cao2020_20211029_121303.xlsx',
 <http.client.HTTPMessage at 0x23297829ca0>)

<br>

#### Load Spreadsheet using pandas

***

In [11]:
# Download and parse the excel spreadsheet.
df = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', skiprows=10)

In [ ]:
df

In [13]:
# Spot check a random row.
df.iloc[753]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Road Transport Technology and Management
COURSE CODE2                                                           LC286
R1 POINTS                                                                264
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      264
EOS Random *                                                             NaN
EOS Mid-point                                                            360
LEVEL                                                                      7
HEI                                         Limerick Institute of Technology
Test/Interview #                                                         NaN

In [14]:
# Spot check the last row.
df.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [15]:
# Create a file path for the pandas data.
path = 'data/cao2020_' + nowstr + '.csv'

In [16]:
# Save pandas data frame to disk.
df.to_csv(path)

<br>

## 2019 Points

https://www.cao.ie/index.php?page=points&p=2019

***

##### Steps to reproduce

**NB: for the video I truncated the CSV file to save time.**

1. Download original pdf file.
2. Open original pdf file in Microsoft Word.
3. Save Microsoft Word's converted PDF in docx format.
4. Re-save Word document for editing.
5. Delete headers and footers.
6. Delete preamble on page 1.
7. Select all and copy.
8. Paste into Notepad++.
9. Remove HEI name headings and paste onto each course line.
10. Delete blank lines.
11. Replaced double tab charaters with a single. (3 occurences).
12. Deleted tab charater at end of line 308.
12. Change backticks to apostrophes.

In [22]:
df2019 = pd.read_csv('data/cao2019_20211029_113930_edited.csv', sep='\t')

In [23]:
df2019

,HEI,Course Code,INSTITUTION and COURSE,EOS,Mid
0,Athlone Institute of Technology,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,Athlone Institute of Technology,AL802,Software Design with Cloud Computing,301,306.0
2,Athlone Institute of Technology,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,Athlone Institute of Technology,AL805,Network Management and Cloud Infrastructure,329,442.0
4,Athlone Institute of Technology,AL810,Quantity Surveying,307,349.0
...,...,...,...,...,...
385,Technological University Dublin Blanchardstown...,BN118,Early Childhood Care and Education,290,320.0
386,Technological University Dublin Blanchardstown...,BN119,Digital Marketing,270,308.0
387,Technological University Dublin Blanchardstown...,BN120,Digital Forensics and Cyber Security,281,326.0
388,Technological University Dublin Blanchardstown...,BN121,Mechatronic Engineering,276,340.0


***

## End